# 📘 LLM 연동 및 프롬프트 엔지니어링 기초
주제: OpenAI 및 Hugging Face 연동 환경 구축 & 제조 특화 프롬프트 설계 실습

### Part 1. [사전 준비] Hugging Face 토큰 발급 가이드

1. Hugging Face란?
- "AI계의 깃허브(GitHub)"입니다. 전 세계의 오픈소스 모델과 데이터셋이 모여있는 곳으로, 우리가 나중에 RAG를 하거나 특수한 제조 데이터를 찾을 때 반드시 필요합니다.

2. 회원가입 및 토큰 발급 절차 (Step-by-Step)
1) 접속: https://huggingface.co/ 접속 후 우측 상단 [Sign Up] 클릭하여 회원가입.
2) 설정 이동: 로그인 후 우측 상단 프로필 아이콘 클릭 -> [Settings] 메뉴 선택.
3) 토큰 메뉴: 좌측 메뉴 리스트에서 [Access Tokens] 클릭.
4) 토큰 생성:
- [Create New Token] 버튼 클릭.
- Token type: Fine-grained 또는 Write 선택 (실습용으로는 Write 권장).
- Name: Manufacturing_Lab 등 식별하기 쉬운 이름 입력.
- [Create token] 클릭.
5) 복사: 생성된 토큰 값(hf_...로 시작)을 복사하여 메모장에 잘 보관. (한 번 창을 닫으면 다시 볼 수 없으니 주의!)

### Part 2. [실습] LLM 연동 및 프롬프트 엔지니어링 (Colab Notebook)

실습 환경 구축
1. 라이브러리 설치 및 API Key 설정
- LLM 실습을 위해 필요한 핵심 라이브러리들을 설치합니다.

- openai: GPT 모델 사용을 위한 공식 라이브러리

- langchain: LLM 애플리케이션 개발을 돕는 프레임워크

- huggingface_hub: 허깅페이스 연동 라이브러리

In [2]:
# 1. 필수 라이브러리 설치
!pip install -q openai langchain langchain-community langchain-core huggingface_hub

import os
import getpass
from huggingface_hub import login

# 2. API Key 설정 (보안을 위해 getpass 사용)
# 강사님이 제공해주신 OpenAI Key를 입력하세요.
print("=== OpenAI API Key 입력 ===")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key를 입력하세요 (sk-...): ")

# 본인이 발급받은 Hugging Face Token을 입력하세요.
print("\n=== Hugging Face Token 입력 ===")
hf_token = getpass.getpass("Hugging Face Token을 입력하세요 (hf_...): ")
login(token=hf_token)

print("\n✅ 모든 인증이 완료되었습니다!")

=== OpenAI API Key 입력 ===
OpenAI API Key를 입력하세요 (sk-...): ··········

=== Hugging Face Token 입력 ===
Hugging Face Token을 입력하세요 (hf_...): ··········

✅ 모든 인증이 완료되었습니다!


### Hello World (기본 연동)
2. Hello LLM! (기본 연결 테스트)
- 가장 기본적인 대화를 시도해 봅니다.
- gpt-3.5-turbo (또는 gpt-4o) 모델에게 간단한 인사를 건네고 응답을 받아옵니다.

In [3]:
from openai import OpenAI

client = OpenAI()

# 간단한 질문 던지기
response = client.chat.completions.create(
    model="gpt-3.5-turbo", # 실습 비용 절감을 위해 3.5 사용 (gpt-4o 권장)
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕? 제조 현장에서 디지털 트윈이 왜 중요한지 한 문장으로 설명해줘."}
    ]
)

print(f"🤖 AI 응답: {response.choices[0].message.content}")

🤖 AI 응답: 디지털 트윈은 제조 현장에서 실시간 상태 모니터링과 예측 분석을 가능하게 해줘서 생산 효율을 높이는데 중요하다.


### 프롬프트 엔지니어링 - 페르소나
3. 프롬프트 엔지니어링: 페르소나(Persona) 부여
- LLM에게 **역할(Role)** 을 부여했을 때와 그렇지 않았을 때의 답변 품질 차이를 비교합니다. 제조 현장의 '안전 관리자' 역할을 부여해 보겠습니다.

- Bad Prompt: 맥락 없이 질문만 던짐.

- Good Prompt (Persona): 역할, 상황, 답변 형식을 구체적으로 지시.

In [4]:
# 시나리오: 화학 물질 누출 사고 발생

# [Case 1] 나쁜 프롬프트 예시
bad_prompt = "공장에 화학 물질이 샜어. 어떻게 해?"

print("--- [Bad Prompt 결과] ---")
response_bad = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": bad_prompt}]
)
print(response_bad.choices[0].message.content)


# [Case 2] 좋은 프롬프트 예시 (페르소나 부여)
system_persona = """
당신은 반도체 제조 공장의 '최고 안전 책임자(CSO)'입니다.
긴급 상황 발생 시, 작업자의 생명 보호를 최우선으로 하며, 국제 안전 규격(OSHA/ISO)에 의거하여 답변해야 합니다.
답변은 반드시 다음 형식으로 작성하세요:
1. 🚨 위험 등급 판단
2. 🏃‍♂️ 즉각적인 대피 및 차단 절차 (3단계)
3. 📞 신고 및 보고 체계
"""

good_prompt = "현재 에칭 공정(Etching Process) 라인에서 '불산(HF)' 가스 누출 경보가 울리고 있습니다. 작업자들이 패닉 상태입니다."

print("\n--- [Good Prompt 결과 (Persona 적용)] ---")
response_good = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_persona},
        {"role": "user", "content": good_prompt}
    ]
)
print(response_good.choices[0].message.content)

--- [Bad Prompt 결과] ---
화학 물질이 샌 공장에서는 즉시 대응해야 합니다. 먼저 화재나 폭발 위험이 있는지 확인하고, 즉시 화재를 진압하거나 필요 시 주변 지역을 대피시켜야 합니다. 

그리고 새어 나온 화학 물질의 종류와 양을 파악하여 적절한 방법으로 처리해야 합니다. 이때는 화학물질 안전 데이터 시트를 참고하여 적절한 대응 방법을 결정해야 합니다.

그리고 화학 물질이 새어 나온 지역을 폐쇄하고 오염된 지역을 정화하는 작업이 필요합니다. 이때는 보호장비를 착용하고 전문가의 도움을 받아야 합니다. 

또한 화학 물질이 새어 나온 사실을 관할 관리 기관에 신고하여 사고 조사와 재발 방지 대책을 마련해야 합니다.화학 물질 유출 사고는 심각한 사고로 이어질 수 있으므로 빠른 대응이 필요합니다.

--- [Good Prompt 결과 (Persona 적용)] ---
1. 🚨 위험 등급 판단
   - 위급 상황: 불산(HF) 가스 누출
   - 위험 등급: 심각 (긴급 대응 필요)

2. 🏃‍♂️ 즉각적인 대피 및 차단 절차 (3단계)
   가. 작업자들에게 즉시 "비상 상황, 불산(HF) 가스가 누출 중. 대피 및 안전장비 착용!"을 외치고 대피 시작을 명령합니다.
   나. 가스 누출이 발생한 구역 주변의 공기 이종화된 구역으로 대피 시키고, 가스 누출 발생 지점의 차단 밸브를 닫아 더 이상의 유출 방지합니다.
   다. 대피한 작업자들을 안전한 장소로 이송시키고, 응급상황 발생 시를 대비하여 응급 구급대원을 호출하여 지시합니다.

3. 📞 신고 및 보고 체계
   - 불산(HF) 가스 누출 사고를 즉시 공장 내부의 비상전화번호를 통해 119에 신고합니다. 
   - 동시에 해당 사고를 사내 안전팀 및 상급자에 빠르게 보고하여 상황을 전파하고 추가 지시를 받습니다. 
   - 사고 발생 원인의 조사 및 사고 관리 절차를 따르며, 사고 발생에 대한 보고와 대응 과정을 기록하여 사후 조치 및 예방에 활용합니다.


### 프롬프트 엔지니어링 - CoT
4. CoT (Chain of Thought): 생각의 사슬
- 복잡한 설비 문제를 해결할 때, AI가 **단계적으로 생각(Step-by-Step)** 하도록 유도하여 **환각(Hallucination)** 을 줄이고 논리적인 답변을 얻는 기법입니다.

- Mission: 모터의 이상 소음 원인을 단계별로 추론하라.

In [5]:
# 복잡한 진단 요청
user_query = """
3호기 컨베이어 모터에서 '끼이익' 하는 고주파 소음이 불규칙하게 들리고,
모터 표면 온도가 평소보다 15도 높습니다. 진동 수치는 정상 범위입니다.
원인이 무엇이고 어떻게 해야 합니까?
"""

# CoT(Chain of Thought) 프롬프트 적용
cot_system_prompt = """
당신은 설비 진단 전문가입니다.
사용자의 질문에 대해 바로 결론을 내리지 말고, 다음 단계에 따라 논리적으로 생각한 뒤 답변하세요(Chain of Thought).

[생각의 단계]
1. 증상 분석: 사용자가 제시한 증상(소음, 온도, 진동)을 각각 분석하십시오.
2. 가설 수립: 진동은 정상이지만 소음과 발열이 발생하는 모순적인 상황에 대해 가능한 기술적 원인을 나열하십시오.
3. 검증 방법: 각 가설을 확인하기 위해 현장에서 무엇을 체크해야 하는지 기술하십시오.
4. 최종 결론: 가장 유력한 원인과 조치 사항을 요약하십시오.
"""

print("--- [CoT 분석 결과] ---")
response_cot = client.chat.completions.create(
    model="gpt-3.5-turbo", # 추론 능력이 더 필요한 경우 gpt-4 권장
    messages=[
        {"role": "system", "content": cot_system_prompt},
        {"role": "user", "content": user_query}
    ]
)
print(response_cot.choices[0].message.content)

--- [CoT 분석 결과] ---
[생각의 단계]
1. 증상 분석:
- 소음: 고주파 소음이 불규칙하게 발생
- 온도: 모터 표면 온도가 15도 높음
- 진동: 정상 범위

2. 가설 수립:
- 소음과 발열이 발생하는 모순적인 상황에서 가능한 원인들:
  - 베어링 손상: 베어링부의 마모나 손상으로 인해 소음 발생 및 열이 발생할 수 있음
  - 코일 감근: 모터 코일 간겼거나 절연불량으로 인한 고주파 소음 발생 및 열 증가
  - 블로워 문제: 냉각 장치인 블로워의 고장으로 열이 증가하여 모터 온도가 상승

3. 검증 방법:
- 베어링 손상 확인: 베어링 볼의 회전 여부 및 마모 상태 확인
- 코일 감근 상태 확인: 코일 간근 여부 및 절연 상태 검토
- 블로워 작동 확인: 냉각 팬 또는 블로워 작동 여부 확인

4. 최종 결론:
- 가장 가능성이 높은 원인은 베어링 손상으로 인한 것으로 판단됩니다.
- 조치:
  - 베어링 교체: 손상된 베어링을 새로 교체하여 소음과 발열 문제 해결
  - 모터 내부 청소: 베어링 교체 후 모터 내부의 먼지 등을 청소하여 원활한 운전 환경 조성
- 모터가 정상적으로 작동할 수 있도록 전문가에게 점검 및 수리를 의뢰하는 것이 좋습니다.


### Hugging Face 활용 (맛보기)
5. Hugging Face 데이터셋 확인
- 우리가 연결한 Hugging Face가 잘 작동하는지 확인하기 위해, 제조 관련 데이터셋 정보를 코드로 조회해 봅니다. 이는 나중에 우리가 자체적인 RAG 시스템을 구축할 때 외부 지식을 가져오는 통로가 됩니다.

In [6]:
from huggingface_hub import list_datasets

# Hugging Face Hub에서 'manufacturing' 관련 데이터셋 검색 (상위 5개)
print("=== Hugging Face Manufacturing Datasets 검색 ===")
datasets = list_datasets(search="manufacturing", limit=5)

for i, dataset in enumerate(datasets):
    print(f"{i+1}. {dataset.id} (Likes: {dataset.likes})")

print("\n✅ Hugging Face 연결 상태 양호! 이후 실습에서 모델 및 데이터를 자유롭게 불러올 수 있습니다.")

=== Hugging Face Manufacturing Datasets 검색 ===
1. akumar33/manufacturing (Likes: 1)
2. g4ndh1/Additive-Manufacturing-Digital-Twin (Likes: 3)
3. abhishekchoudhary0509/manufacturing_plant (Likes: 0)
4. BAAI/IndustryCorpus2_other_manufacturing (Likes: 4)
5. g3lu/additive_manufacturing_questions-R1 (Likes: 2)

✅ Hugging Face 연결 상태 양호! 이후 실습에서 모델 및 데이터를 자유롭게 불러올 수 있습니다.


# 📘 실습: 제조 특화 프롬프트 엔지니어링
주제: SOP(표준 운영 절차) 주입 및 비정형 로그 데이터의 정보 추출

### 실습 개요
1. System Prompt의 힘: 지식 주입하기 (In-Context Learning)
- LLM은 우리 공장의 **SOP(표준 운영 절차)** 를 모릅니다. 하지만, 우리가 **System Prompt(시스템 프롬프트)** 에 SOP 내용을 '맥락(Context)'으로 넣어주면, 마치 그 매뉴얼을 외운 전문가처럼 답변할 수 있습니다. 이것이 RAG의 가장 기초가 되는 In-Context Learning입니다.

- 시나리오: 사출 성형기에서 ERR-707이라는 에러가 발생했습니다. 일반 챗GPT는 이 코드를 모르지만, SOP를 주입받은 AI는 정확한 조치 방법을 안내합니다.

- [예상 답변] 매뉴얼에 따르면 ERR-707 에러에서 히터 밴드 저항이 20~30옴 사이(25옴)라면 히터는 정상입니다. 따라서 원인은 SSR(Solid State Relay) 고장일 가능성이 높습니다.조치 방법: 제어반 내 SSR을 교체하십시오. 교체 후 반드시 **오토 튜닝(Auto Tuning)** 을 실시하십시오.

In [8]:
import os
import getpass
from openai import OpenAI

client = OpenAI()

# 1. 우리 공장의 가상 SOP (표준 운영 절차) 정의
# 이 내용은 챗GPT 학습 데이터에 없는 우리 회사만의 지식입니다.
my_factory_sop = """
[사출 성형기 에러 대응 매뉴얼 Ver 2.0]

1. 에러 코드: ERR-707 (노즐 온도 편차 과다)
   - 현상: 설정 온도와 실제 온도가 ±10도 이상 차이 남.
   - 원인: 히터 밴드 단선 또는 SSR(Solid State Relay) 고장.
   - 조치 순서:
     1) 제어 패널에서 히터 전원 OFF.
     2) 멀티미터로 히터 밴드 저항 측정 (정상 범위: 20~30옴).
     3) 저항이 무한대면 히터 교체, 정상이면 제어반 내 SSR 교체.
     4) 교체 후 오토 튜닝(Auto Tuning) 실시.

2. 에러 코드: ERR-808 (형체력 부족)
   - 현상: 금형이 완전히 닫히지 않고 Flash(Burr) 발생.
   - 조치: 토글 링크의 구리스 급유 상태 확인 및 형체 실린더 압력 재설정.
"""

# 2. System Prompt 설계 (SOP 주입 + 페르소나)
system_prompt_sop = f"""
당신은 사출 공정의 '설비 보전 전문가'입니다.
아래 제공된 [대응 매뉴얼]을 숙지하고, 작업자의 질문에 대해 매뉴얼에 근거하여 정확한 조치 방법을 안내하세요.
매뉴얼에 없는 내용은 '매뉴얼에 해당 내용이 없습니다'라고 답해야 합니다.

{my_factory_sop}
"""

# 3. 사용자 질문 (상황 발생)
user_question = "지금 3호기에서 ERR-707 알람이 떴어. 히터 저항을 재봤는데 25옴이 나오네. 어떻게 해야 해?"

# 4. LLM 호출
response = client.chat.completions.create(
    model="gpt-4o", # 또는 gpt-3.5-turbo
    messages=[
        {"role": "system", "content": system_prompt_sop},
        {"role": "user", "content": user_question}
    ],
    temperature=0 # 매뉴얼대로 정확히 답해야 하므로 창의성(온도)을 0으로 설정
)

print("=== [AI 보전 전문가의 답변] ===")
print(response.choices[0].message.content)

=== [AI 보전 전문가의 답변] ===
히터 저항이 25옴으로 정상 범위(20~30옴) 내에 있으므로, 히터는 정상입니다. 따라서 제어반 내 SSR(Solid State Relay)을 교체해야 합니다. 교체 후에는 오토 튜닝(Auto Tuning)을 실시하세요.


### 비정형 데이터 정보 추출
2. 비정형 데이터(Log)를 정형 데이터(JSON)로 변환
- 현장 작업자들은 작업 일지를 "오후 2시에 모터가 멈춰서 필터 갈았음"처럼 편한 말로 적습니다. 이런 비정형 텍스트는 엑셀로 통계를 낼 수가 없습니다.

- LLM을 이용하여 이 텍스트에서 **[일시, 설비명, 고장 원인, 조치 내용]** 을 뽑아내어 컴퓨터가 이해할 수 있는 JSON 형식으로 변환해 보겠습니다.

In [9]:
# 1. 현장의 지저분한(Unstructured) 작업 일지 예시
raw_logs = """
[김반장 작업일지]
- 오늘 10시 반쯤에 A라인 컨베이어 벨트가 찢어져서 라인 섰음. 창고에서 예비 벨트 가져와서 11시에 교체 완료함.
- 오후 2시 15분, 2호기 유압 펌프에서 기름이 새는 거 발견. 오링이 낡아서 새 거로 갈아끼움.
- 4시경에 포장기 센서 에러 떴는데 그냥 먼지 털어내니까 다시 됨. 별다른 부품 교체는 안 함.
"""

# 2. 정보 추출(Information Extraction)을 위한 프롬프트 설계
extraction_prompt = """
당신은 '데이터 엔지니어'입니다.
작업자가 작성한 비정형 작업 일지를 읽고, 중요한 정보를 추출하여 **JSON 데이터 형식**으로 변환하세요.

[추출 규칙]
- 시간: 작업이 발생한 대략적인 시각 (YYYY-MM-DD HH:MM 형식, 오늘은 2026-01-10로 가정)
- 설비: 문제가 발생한 설비 이름
- 문제: 발생한 문제나 증상
- 조치: 수행한 조치 내용
- 부품교체여부: 부품을 교체했으면 'Y', 아니면 'N'

반드시 JSON 형식만 출력하세요. 설명은 필요 없습니다.
"""

# 3. LLM 호출
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": extraction_prompt},
        {"role": "user", "content": raw_logs}
    ],
    temperature=0
)

# 4. 결과 확인
import json

try:
    extracted_data = json.loads(response.choices[0].message.content)
    print("=== [JSON 변환 결과] ===")
    print(json.dumps(extracted_data, indent=4, ensure_ascii=False))
except:
    print("JSON 파싱 실패, 원본 출력:")
    print(response.choices[0].message.content)

=== [JSON 변환 결과] ===
{
    "작업일지": [
        {
            "시간": "2026-01-10 10:30",
            "설비": "A라인 컨베이어",
            "문제": "벨트 찢어짐",
            "조치": "예비 벨트로 교체",
            "부품교체여부": "Y"
        },
        {
            "시간": "2026-01-10 14:15",
            "설비": "2호기 유압 펌프",
            "문제": "기름 누출",
            "조치": "오링 교체",
            "부품교체여부": "Y"
        },
        {
            "시간": "2026-01-10 16:00",
            "설비": "포장기 센서",
            "문제": "에러 발생",
            "조치": "먼지 제거",
            "부품교체여부": "N"
        }
    ]
}


### 자동 요약 및 보고서 생성
3. 작업 일지 요약 및 교대 근무자 인계 리포트 작성
- 하루 동안 쌓인 수십 건의 로그를 다음 근무자가 다 읽을 수는 없습니다. LLM에게 **"오늘의 핵심 이슈 요약(Summarization)"** 을 요청하여, **Shift Report(교대 근무 보고서)** 를 자동으로 작성하게 합니다.

In [10]:
# 1. 요약(Summarization) 프롬프트 설계
summary_prompt = """
당신은 공장의 '교대 근무 팀장'입니다.
위에서 추출된 작업 내역을 바탕으로, 다음 근무자에게 전달할 '일일 설비 이슈 리포트'를 작성하세요.

[작성 가이드]
1. 오늘 발생한 총 건수와 부품 교체 건수를 상단에 요약할 것.
2. 가장 심각해 보이는 이슈 1가지를 선정하여 '중점 관리 항목'으로 강조할 것.
3. 톤앤매너: 간결하고 명확한 보고서체 (~함, ~임) 사용.
"""

# 앞서 LLM이 추출했던 내용을 다시 입력으로 줌 (Chaining)
extracted_content = response.choices[0].message.content

# 2. LLM 호출
summary_response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": summary_prompt},
        {"role": "user", "content": f"오늘의 작업 내역 JSON 데이터:\n{extracted_content}"}
    ]
)

print("=== [Shift Handover Report] ===")
print(summary_response.choices[0].message.content)

=== [Shift Handover Report] ===
**일일 설비 이슈 리포트**

- 오늘 발생한 총 이슈 건수: 3건
- 부품 교체 건수: 2건

**중점 관리 항목**
- 설비: 2호기 유압 펌프
  - 문제: 기름 누출
  - 조치: 오링 교체
  - 부품 교체: 예
  - 설명: 유압 펌프의 기름 누출은 생산 중단 및 안전 문제를 유발할 수 있음. 이로 인한 추가적인 점검과 예방 조치 필요함.

**기타 이슈 내역**
1. 설비: A라인 컨베이어
   - 문제: 벨트 찢어짐
   - 조치: 예비 벨트로 교체
   - 부품 교체: 예

2. 설비: 포장기 센서
   - 문제: 에러 발생
   - 조치: 먼지 제거
   - 부품 교체: 아니오

각 설비의 문제 원인을 파악하고, 반복적인 문제가 발생하지 않도록 주의 바람. 추가 점검 필요 시 연락 요망.


💡 실습 마무리
1. 시스템 프롬프트의 중요성 (Case 1):

- "system 메시지에 SOP를 넣는 순간, 이 AI는 인터넷에 떠도는 일반론을 말하는 게 아니라 우리 공장의 규칙을 따르는 전문가가 됩니다."

2. 데이터의 가치 창출 (Case 2):

- "현장에 널려 있는 텍스트 로그들은 그동안 '버려지는 데이터'였습니다. 하지만 LLM을 거치면 **통계 분석이 가능한 엑셀 데이터(JSON)** 로 되살아납니다."

3. 업무 효율화 (Case 3):

- "퇴근할 때마다 일보 쓰느라 30분씩 걸리셨죠? AI를 쓰면 3초 만에 초안을 써줍니다. 사람은 확인만 하면 됩니다."